<a href="https://colab.research.google.com/github/hatopopvr/MyBSTile/blob/main/MyBSTile_En.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MyBSTile_En
<b>Data</b>
- Score Data form ScoreSaber Public API - [doc](https://docs.scoresaber.com/)  
- Cover Image form - https://cdn.scoresaber.com/covers/{hash}.png  

<b>Author</b>
- hatopop ([@hatopop_vr](https://twitter.com/hatopop_vr))

In [3]:
#@title <h1>Input & GetData from ScoreSaber</h1> { display-mode: "form" }
#@markdown #### please input information
#@markdown ---
#@markdown <font size="2">`player_id` : PlayerID for ScoreSaber. <b>Must</b> be changed.<br/></font>
player_id =  76561198412839195#@param {type:"number"}

#markdown ---
#markdown <h4>Setting</h4>
#markdown <font size="2">`ss_plus_is_enable`: AccRank区分にSS+を使用するか。☑で使用。  </font><br />
ss_plus_is_enable = True #param {type:"boolean"}
#markdown <font size="2">`ss_plus_val`:SS+の設定値です。必要に応じて変更ください。範囲は91-99。
ss_plus_val = 95 #param {type:"slider", min:91, max:99, step:1}
#@markdown <font size="2">`score_count`:The number of scores to retrieve from the ScoreSaber API.
score_count = 40 #@param {type:"integer"}

#@markdown <font size="2">`recent_range`:Range of days to be filterd in `recent`
recent_range = 1 #@param {type:"integer"}
latest = recent_range
#@markdown <font size="2">`view_score_type`:The target pages to be retrieved by ScoreSaber API. `top`:`Top Scores`, `recent`:`Recent Scores`
view_score_type = "recent" #@param ["top", "recent"]

#@markdown ---

#@markdown <b>Target Setting</b>
#@markdown <font size=2> : Target value. Highlight or filter.</font>
target_global_rank = 50 #@param {type:"slider", min:0, max:5000, step:1}
target_pp = 270 #@param {type:"slider", min:0, max:600, step:10}
target_weight = 1 #@param {type:"slider", min:0, max:600, step:1}


# ----------------------------------------
# Import Library
# ----------------------------------------
!pip install numpy==1.23.5
!pip install pandas==1.5.0
import warnings
warnings.filterwarnings("ignore")
import os
import shutil
import pandas as pd
from pandas import json_normalize
import json
from datetime import datetime, timedelta
from dateutil import tz
import time
from tqdm import tqdm
import requests
import math
import gc
from IPython.display import HTML, Javascript,Image
import numpy as np
from PIL import Image
import copy

# ----------------------------------------
# Time Zone Set
# ----------------------------------------
#@title Timezone UTC->日本時間+9:00での実行日時の取得(tz_ja)
tz_ja = pd.Timestamp(datetime.now()).tz_localize('UTC').tz_convert('Asia/Tokyo')

# ----------------------------------------
# Parameter Setting
# ----------------------------------------
# SS_plus設定値
ss_plus = "SS+{}".format(ss_plus_val)
ss_plus_rate = "SS+{}-Rate".format(ss_plus_val)
# その他 colab表示など設定
pd.options.display.precision = 2
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# 出力最大高さ
def resize_colab_cell():
    display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))

get_ipython().events.register('pre_run_cell', resize_colab_cell)

# ----------------------------------------
# Function Color Style Setting
# ----------------------------------------
# Difficulty color
def color_difficulty(val):
    if val == "Easy":
        return 'color: #66BB6A; font-weight: bold;'
    elif val == "Normal":
        return 'color: #29B6F6; font-weight: bold;'
    elif val == "Hard":
        return 'color: #FB8C00; font-weight: bold;'
    elif val == "Expert":
        return 'color: #E53935; font-weight: bold;'
    elif val == "Expert+":
        return 'color: #8E24AA; font-weight: bold;'
    elif val == "ExpertPlus":
        return 'color: #8E24AA; font-weight: bold;'
    return 'color: #black'

# Acc Rank color
def color_acc_rank(val):
    if val == "SSS":
        return 'color: #00ffff; font-weight: bold;'
    elif ss_plus_is_enable and val == ss_plus:
        return 'color: #636EFA; font-weight: bold;'
    elif val == "SS":
        return 'color: #ff4500; font-weight: bold;'
    elif val == "S":
        return 'color: #ffaaff; font-weight: bold;'
    elif val == "A":
        return 'color: #ffcc66; font-weight: bold;'
    elif val == "B":
        return 'color: #cccc66; font-weight: bold;'
    else:
        return 'color: #666666; font-weight: bold;'

# FC color
def color_fc(val):
    if val == "FC":
        return 'color: #23D160; font-weight: bold;'
    else:
        return 'color: black'

# pp color
def color_pp(val):
    if val >= target_pp:
        color = "red"
    else:
        color = "black"
    return 'color: %s' % color

# rank color
def color_rank(val):
    if val <= target_global_rank:
        color = "red"
    else:
        color = "black"
    return 'color: %s' % color

# latest color
def color_latest(x):
    if  x == 1:
        color = "red"
    else:
        color = "black"
    return 'color: %s' % color

# weight
def format_weight(val):
    if val <= target_weight:
        return "".format(val)
    else:
        return "({:.1f}pp)".format(val)

# ----------------------------------------
# Function Pandas Apply
# ----------------------------------------
def func_mode(x):
    if  x == "SoloStandard":
        return "Standard"
    else:
        return x

def func_unrank(x):
    if x == 0:
        return 0
    else:
        return x

def func_accrank_format(x):
    if  x > 100:
        return "-"
    elif  x == 100:
        return "SSS"
    elif x >= ss_plus_val and ss_plus_is_enable:
        return ss_plus
    elif x >= 90:
        return "SS"
    elif x >= 80:
        return "S"
    elif x >= 65:
        return "A"
    elif x >= 50:
        return "B"
    elif x >= 35:
        return "C"
    elif x >= 20:
        return "D"
    elif x >= 0:
        return "E"
    else:
        return "-"

def func_difficulty_format(val):
    if val == "Easy":
        return 'E'
    elif val == "Normal":
        return 'N'
    elif val == "Hard":
        return 'H'
    elif val == "Expert":
        return 'Ex'
    elif val == "Expert+":
        return 'Ex+'
    elif val == "ExpertPlus":
        return 'Ex+'
    return '-'

def func_fc_format(val):
    if val == "FC" or val:
        return 'FC'
    else:
        return ''

def func_latest(x):
    if  x <= latest:
        return 1
    else:
        return 0

# weight
def format_weight(val):
    if val <= target_weight:
        return "".format(val)
    else:
        return "({:.1f}pp)".format(val)

# star
def format_star(val):
    if val > 0:
        return "{:.2f}★".format(val)
    else:
        return ""
    return "{:.2f}★".format(val)

# acc
def format_acc(val):
    if val <= 100:
        return "{:.2f}%".format(val)
    else:
        return ""
    return "{:.2f}%".format(val)

# pp
def format_pp(val):
    if val > 0:
        return "{:.1f}pp".format(val)
    else:
        return ""
    return "{:.1f}pp".format(val)

# ----------------------------------------
# Player Image Css
# ----------------------------------------

def func_tile_css(_width=1000,
             _cover_image_size=200,
             _star_font_size=22,
             _diff_font_size=18,
             _acc_font_size=18,
             _fc_font_size=18,
             _pp_font_size=22,
             _weight_font_size=8,
             _rank_font_size=22,
             _date_font_size=18,
             _player_font_size=22
             ):
    return """
        <style>
        .flex_box {
            width:""" + str(_width) + """px;
            display: flex; /* フレックスボックスにする */;
            flex-wrap: wrap;
            padding:0px;
            margin:0px;
        }
        .flex_item {
            display: inline-block;
            position: relative;
            padding:0px 5px 0px 0px;
            margin:0px;
        }
        .flex_str_top, .flex_str_bottom_left, .flex_str_bottom_right{
            position: absolute;
            font-weight: bold;
            white-space:nowrap;
            overflow:hidden;
            color: #000;
            width:""" + str(_cover_image_size-10) + """px;
            text-shadow:1px 1px 0 #FFF, -1px -1px 0 #FFF,
                    -1px 1px 0 #FFF, 1px -1px 0 #FFF,
                    0px 1px 0 #FFF,  0-1px 0 #FFF,
                    -1px 0 0 #FFF, 1px 0 0 #FFF;
        }

        .flex_str_top {
            left: 5px;
            top: 5px;
        }

        .flex_str_bottom_left {
            left: 5px;
            bottom: 5px;
        }

        .flex_str_bottom_right {
            left: 5px;
            bottom: 5px;
        }

        .flex_star, .flex_acc, .flex_accrank, .flex_fc ,.flex_pp, .flex_weight, .flex_rank {
            padding:0px;
            margin:0px;
        }

        .flex_star{
            font-size: """ + str(_star_font_size) + """px;
        }

        .flex_diff{
            font-size: """ + str(_diff_font_size) + """px;
        }

        .flex_acc{
            font-size: """ + str(_acc_font_size) + """px;
        }

        .flex_fc {
            font-size: """ + str(_fc_font_size) + """px;
        }

        .flex_pp{
            font-size: """ + str(_pp_font_size) + """px;
        }

        .flex_weight{
            font-size: """ + str(_weight_font_size) + """px;
        }

        .flex_rank {
            font-size: """ + str(_rank_font_size) + """px;
            text-align:right;
        }
        .flex_date {
            font-size: """ + str(_date_font_size) + """px;
            text-align:right;
        }
        </style>
        """

# ----------------------------------------
# Get Score Data from ScoreSaber
# ----------------------------------------
def get_range_count(_score_count, _page_count):
    return int(_score_count / _page_count)

page_count = 100
range_count = get_range_count(score_count, page_count)


# url = r"https://scoresaber.com/api/player/{}/scores?sort={}&limit={}".format(player_id, view_score_type, score_count)
# response = requests.get(url)
# res_data = response.json()
# df_scores = json_normalize(res_data['playerScores'])

# url = r"https://scoresaber.com/api/player/{}/scores?sort=recent".format(player_id)
# url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&limit={}".format(player_id, page_count)
url = r"https://scoresaber.com/api/player/{}/scores?sort={}&limit={}".format(player_id, view_score_type, page_count)
response = requests.get(url)
res_data = response.json()
df_scores = json_normalize(res_data['playerScores'])

if range_count > 0:
    for i in tqdm(range(range_count)):
        #url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&page={}&limit={}".format(player_id, i + 2, page_count)
        url = r"https://scoresaber.com/api/player/{}/scores?sort={}&page={}&limit={}".format(player_id, view_score_type, i + 2, page_count)
        try:
            response = requests.get(url)
            res_data = response.json()
            df_scores=df_scores.append(json_normalize(res_data['playerScores']), ignore_index=True)
        except:
            break

df_scores = df_scores[:score_count]

if len(df_scores) > 0:

    df_scores['Song'] = df_scores['leaderboard.songName'] + " " + df_scores['leaderboard.songSubName'] + " / " + df_scores['leaderboard.songAuthorName'] + " [" + df_scores['leaderboard.levelAuthorName'] + "]"
    df_scores['SongName'] = df_scores['leaderboard.songName']
    df_scores['SongSub'] = df_scores['leaderboard.songSubName']
    df_scores['SongAuthor'] = df_scores['leaderboard.songAuthorName']
    df_scores['LevelAuthor'] = df_scores['leaderboard.levelAuthorName']
    df_scores['Hash'] = df_scores['leaderboard.songHash'].str.upper()
    df_scores['Acc'] = df_scores['score.modifiedScore'] / df_scores['leaderboard.maxScore'] * 100
    df_scores['MaxScore'] = df_scores['leaderboard.maxScore']

    df_scores['Mode'] = df_scores['leaderboard.difficulty.gameMode'].apply(func_mode)
    _df_scores = df_scores['leaderboard.difficulty.difficultyRaw'].str.split('_', expand=True)
    _df_scores.columns = ['_','Difficulty', 'Mode']
    df_scores['Difficulty'] = _df_scores['Difficulty']
    df_scores['Stars'] = df_scores['leaderboard.stars']
    df_scores['Level'] = df_scores['Stars'].astype('int')
    df_scores["LevelStr"] = df_scores['Level'].astype('str')
    df_scores['Score'] = df_scores['score.modifiedScore']
    df_scores['Bad'] = df_scores['score.badCuts']
    df_scores['Miss'] = df_scores['score.missedNotes']
    df_scores['Combo'] = df_scores['score.maxCombo']
    df_scores['PP'] = df_scores['score.pp']
    df_scores['PPWeight'] = df_scores['score.pp'] * df_scores['score.weight']
    df_scores['Rank'] = df_scores['score.rank']
    df_scores['Modifiers'] = df_scores['score.modifiers']
    df_scores['Ranked'] = df_scores['leaderboard.ranked']
    df_scores['Qualified'] = df_scores['leaderboard.qualified']
    df_scores['Play'] = df_scores['leaderboard.plays']
    df_scores['DailyPlay'] = df_scores['leaderboard.dailyPlays']
    df_scores['DateUtc'] = pd.to_datetime(df_scores['score.timeSet'])
    _df_scores_idx = df_scores.set_index('DateUtc')
    df_scores['DateJa'] = _df_scores_idx.index.tz_convert('Asia/Tokyo')
    df_scores['Date'] = df_scores['DateJa'].dt.date
    df_scores['Days'] = (tz_ja.date() - df_scores['Date']).dt.days
    df_scores = df_scores.set_index('DateJa')
    df_scores['Months'] = (df_scores['Days'] / 30).astype('int')
    df_scores['DaysStr'] = df_scores['Days'].astype('str')
    df_scores['MonthsStr'] = df_scores['Months'].astype('str')

    df_scores['Latest'] = df_scores['Days'].apply(func_latest)

    df_scores['AccRank'] = df_scores['Acc'].apply(func_accrank_format)

    df_scores['FC'] = df_scores['score.fullCombo'].apply(func_fc_format)

    df_scores = df_scores[[x for x in df_scores.columns if not x.startswith("score.")]]
    df_scores = df_scores[[x for x in df_scores.columns if not x.startswith("leaderboard.")]]

    # 結合前Score情報の抽出(LevelClearedProgress用)
    df_scores_org = df_scores.copy()

    # 改行コード等の除去
    for col in df_scores.columns:
        try:
            if len(df_scores[df_scores[col].str.contains("\n")][[col]]) == 0:
                continue
            else:
                df_scores[col] = df_scores[col].str.replace("\n","")
        except:
            continue

    for col in df_scores.columns:
        try:
            if len(df_scores[df_scores[col].str.contains("\r")][[col]]) == 0:
                continue
            else:
                df_scores[col] = df_scores[col].str.replace("\r","")
        except:
            continue


#@title Player情報の取得--------------------------------------------------
url = r"https://scoresaber.com/api/player/{}/full".format(player_id)
response = requests.get(url)
res_data = response.json()
df_info = json_normalize(res_data)

# df_info["Pic"] = '<img src="'+df_info["profilePicture"]+'" style="width:{}px;"/>'.format(info_image_size)
df_info["TotalPlayCount"] = df_info["scoreStats.totalPlayCount"]
df_info["RankedPlayCount"] = df_info["scoreStats.rankedPlayCount"]

str_player_info = """
-----------[Information]-----------
 Player          : {}
 TotalPlayCount  : {:,.0f}
 RankedPlayCount : {:,.0f}
 ({} results retrieved)
------------------------------------
""".format(df_info["name"][0], df_info["TotalPlayCount"][0], df_info["RankedPlayCount"][0], len(df_scores))

print(str_player_info)

<IPython.core.display.Javascript object>


-----------[Information]-----------
 Player          : hatopop
 TotalPlayCount  : 5,275
 RankedPlayCount : 4,389
 (40 results retrieved)
------------------------------------



# History Tile

In [4]:
#@title ## History Tile

#markdown ---
#markdown <h4>Enable</h4>

history_tile_is_enable = True #param {type:"boolean"}

#@markdown ---
#@markdown <h4>Setting</h4>

data_max_size = 500 #param {type:"slider", min:0, max:5000, step:1}

#@markdown <b>image setting</b>

#@markdown <font size=2> `width` : The width of the frame in px. Set to match (slightly larger than) the size of the image.
width = 1000 #@param {type:"slider", min:200, max:4000, step:100}
#@markdown <font size=2> `cover_image_size` : Width and height in px per image.
cover_image_size = 200 #@param {type:"slider", min:50, max:1000, step:50}

#@markdown <b>font size setting</b>
#@markdown <font size=2> : The setting of each font size.</font>
song_font_size = 22 #@param {type:"slider", min:3, max:50, step:1}
star_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
diff_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
acc_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
fc_font_size = 18 #@param {type:"slider", min:3, max:50, step:1}
pp_font_size = 22 #@param {type:"slider", min:3, max:50, step:1}
weight_font_size = 8 #@param {type:"slider", min:3, max:50, step:1}
rank_font_size = 22 #@param {type:"slider", min:3, max:50, step:1}
date_font_size = 12 #@param {type:"slider", min:3, max:50, step:1}

#@markdown <b>Visible Setting</b>
#@markdown <font size=2> : The setting of show/hide each item.</font>

song_visible = True #@param {type:"boolean"}
star_visible = True #@param {type:"boolean"}
diff_visible = True #@param {type:"boolean"}
acc_visible = True #@param {type:"boolean"}
fc_visible = True #@param {type:"boolean"}
pp_visible = True #@param {type:"boolean"}
weight_visible = True #@param {type:"boolean"}
rank_visible = True #@param {type:"boolean"}
date_visible = True #@param {type:"boolean"}

#@markdown ---
#@markdown <h4>Filters</h4>
#@markdown <font size=2> Each filter function. Keyword filters do not support multiple keywords.</font>
filtered_stars_min = 0 #@param {type:"slider", min:0, max:15, step:1}
filtered_stars_max = 15 #@param {type:"slider", min:0, max:15, step:1}
filtered_pp_min = 0 #@param {type:"slider", min:0, max:600, step:10}
filtered_pp_max = 600 #@param {type:"slider", min:0, max:600, step:10}
filtered_acc_min = 0 #@param {type:"slider", min:0, max:100, step:1}
filtered_acc_max = 100 #@param {type:"slider", min:0, max:100, step:1}
filtered_ranked_is_enable = False #@param {type:"boolean"}
filtered_fullcombo_is_enable = False #@param {type:"boolean"}
song_keyword_filter = "" #@param {type:"string"}

#@markdown ---
#@markdown <h4>Sort</h4>
#@markdown <font size=2> Sort is in descending order by the following four types. PP, DateJa(Date), Acc, Stars</font>
sort_key = "PP" #@param ["PP", "DateJa", "Acc", "Stars"]

def func_flex(x):
    div_song = "<div class='flex_song'>{}</div>".format(x[1]) if song_visible else ""

    span_star = "<span class='flex_star'>{}</span>".format(format_star(x[2])) if star_visible else ""
    span_diff = "<span= class='flex_diff'>{}</span>".format(func_difficulty_format(x[6])) if diff_visible else ""
    div_star_diff = "<div style='{}'>{} {}</div>".format(color_difficulty(x[6]), span_star, span_diff)

    span_acc = "<span class='flex_acc' style='{}'>{}</span>".format(color_acc_rank(x[4]), format_acc(x[3])) if acc_visible else ""
    span_fc = "<span class='flex_fc' style='{}'>{}</span>".format(color_fc(x[5]),func_fc_format(x[5])) if fc_visible else ""
    div_acc_fc = "<div>{} {}</div>".format(span_acc, span_fc)

    span_pp = "<span class='flex_pp' style='{}'>{}</span>".format(color_pp(x[7]), format_pp(x[7])) if pp_visible else ""
    span_weight = "<span class='flex_weight' style='{}'>{}</span>".format(color_pp(x[8]), format_weight(x[8])) if weight_visible else ""
    div_pp_weight = "<div>{} {}</div>".format(span_pp, span_weight)

    filtered_latest_is_enable = True
    if date_visible:
        if view_score_type == "recent":
            div_date = "<div class='flex_date' style='{}'>{}/{} {}:{:02d}</div>".format(color_latest(x[16]), x[12], x[13], x[14], x[15])
        else:
            div_date = "<div class='flex_date' style='{}'>'{}/{}/{}</div>".format(color_latest(x[16]), str(x[11])[2:],x[12], x[13])
    else:
        div_date = ""

    div_rank = "<div class='flex_rank' style='{}'>#{:,.0f}</div>".format(color_rank(x[9]), x[9]) if rank_visible else ""

    span_diff_unrank = "<span= class='flex_star' style='{}'>{}</span>".format(color_difficulty(x[6]), func_difficulty_format(x[6])) if diff_visible else ""
    div_diff_fc = "<div style='{}'>{} {}</div>".format(color_difficulty(x[6]), span_diff, span_fc)

    ranked = x[10]

    if ranked:
        return """
                <div class='flex_item'>{}<br/>
                    <div class='flex_str_top'>
                        {}
                    </div>
                    <div class='flex_str_bottom_left'>
                        {}
                        {}
                        {}
                    </div>
                    <div class='flex_str_bottom_right'>
                        {}
                        {}
                    </div>
                </div>
                """.format(x[0], div_song, div_star_diff, div_acc_fc, div_pp_weight, div_date, div_rank)

    else :
        return """
                <div class='flex_item'>{}<br/>
                    <div class='flex_str_top'>
                        {}
                    </div>
                    <div class='flex_str_bottom_left'>
                        {}
                    </div>
                    <div class='flex_str_bottom_right'>
                        {}
                        {}
                    </div>
                </div>
                """.format(x[0], div_song, div_diff_fc, div_date, div_rank)


df_history = df_scores_org[(1==1)
    #& (df_scores["Modifiers"] != "NF")
    & (df_scores["Stars"] >= filtered_stars_min)
    & (df_scores["Stars"] <= filtered_stars_max)
    & (df_scores["PP"] >= filtered_pp_min)
    & (df_scores["PP"] <= filtered_pp_max)
    & (df_scores["Acc"] >= filtered_acc_min)
    & (df_scores["Acc"] <= filtered_acc_max)
    | (df_scores['Ranked']!=1)
    ]

if view_score_type == "recent":
    df_history = df_history[(df_history["Latest"] == 1)]

if filtered_ranked_is_enable:
    df_history = df_history[(df_history['Ranked']==1)]

if filtered_fullcombo_is_enable:
    df_history = df_history[(df_history['FC']=="FC")]

if len(song_keyword_filter) > 0:
    df_history = df_history[df_history["Song"].str.contains(song_keyword_filter, case=False)]

history_rows_count = len(df_history)

if history_tile_is_enable:
    print("{} results retrieved.\n".format(history_rows_count, data_max_size))
else:
    print("有効フラグが無効です。")

if history_rows_count > 0 and history_tile_is_enable:

    df_history_table = df_history.copy()

    df_history_table['Song'] = "<b>" + df_history_table["SongName"] + " " + df_history_table["SongSub"] + " </b><br /> " + df_history_table["SongAuthor"] + "<br />[" + df_history_table["LevelAuthor"] + "]"

    df_history_table['Year'] = df_history_table.index.year
    df_history_table['Month'] = df_history_table.index.month
    df_history_table['Day'] = df_history_table.index.day
    df_history_table['Hour'] = df_history_table.index.hour
    df_history_table['Minute'] = df_history_table.index.minute

    # df_history_table = df_history_table

    df_history_table['Stars'] = df_history_table['Stars'].apply(func_unrank)

    df_flex_table = df_history_table.copy()

    df_flex_table = df_flex_table.sort_values([sort_key], ascending=False)

    df_flex_table["FlexCover"] = '<img src="https://cdn.scoresaber.com/covers/' + df_flex_table["Hash"] + '.png" style="width:{}px;height:{}px"/>'.format(cover_image_size,cover_image_size)

    df_flex_table["FlexItems"] =df_flex_table[['FlexCover','Song','Stars','Acc','AccRank','FC','Difficulty','PP', 'PPWeight','Rank','Ranked','Year','Month','Day','Hour','Minute','Latest']].apply(func_flex, axis=1)

    flex_head = """
                <div id="imageDIV"
                    class='flex_box_wrapper'>
                    <div class='flex_box'>
                """
    flex_content = " ".join(df_flex_table.reset_index()['FlexItems'].to_list())
    flex_end = "</div></div>"

    img_css = func_tile_css(width,
             cover_image_size,
             star_font_size,
             diff_font_size,
             acc_font_size,
             fc_font_size,
             pp_font_size,
             weight_font_size,
             rank_font_size,
             date_font_size
             )
    html_str = flex_head + flex_content + flex_end + img_css

else:
    html_str = ""

HTML(html_str)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

24 results retrieved.

